## Module INM378/IN3031: Digital Signal Processing and Audio Programming
# Lab 5

### 0. Setup dependencies

In [ ]:
try:
    import google.colab
    import subprocess
    p = subprocess.run(['git', 'rev-parse', '--is-inside-work-tree'], stdout=subprocess.PIPE, universal_newlines=True)
    if p.stdout != 'true\n':
        !git clone --depth 1 -q https://github.com/jpauwels/city_dsp_ap.git
        %cd city_dsp_ap
    else:
        !git pull
except:
    %cd city_dsp_ap

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [ ]:
plt.rcParams['figure.figsize'] = (13,8)

In [ ]:
plt.rcParams['figure.figsize']

### 1. Filter response

Study the phase and frequency response of a filter with coefficients `b = [0.032, -0.053, 0.047, -0.053, 0.032]` and `a = [1, -2.742, 3.735, -2.578, 0.885]`. Use the `scipy.signal` function [`freqz`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqz.html) and adapt the example given in the documentation to create a function `plot_filter(b, a)` that plots the frequency and phase response of a filter given by its coefficients `b` and `a`. Then use that function to check what type of filter this is.

_answer here_

### 2. Filter design

In this part, we're going to analyse energy usage data given in `data/energy_usage.pkl` . The data in the file reflects the energy consumption of a kitchen, reporting a sample per minute.

#### 2.1. Step one

[Unpickle](https://docs.python.org/3.7/library/pickle.html) the data. What is its sampling frequency? How many minutes are recorded?

_answer here_

#### 2.2 Step two

Calculate the power spectrogram in dB of the data using the [`stft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html) function in `scipy.signal`. Try a window size of 1024 samples and larger. Check the documentation for an example on how to visualise the spectrogram. You can adjust the range of the x-axis and y-axis with [`plt.xlim`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xlim.html) and [`plt.ylim`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.ylim.html) respectively. This can be used to zoom in on particular sections of the plot. Remember that the output of the STFT are complex numbers, so you will need to convert it to a power spectrogram in dB yourself. Finally turn your code into a function `plot_spectrogram(time_signal, samplerate)` such that it can be reused later.

What happens at the low frequencies? Can you detect any patterns? At which time in minutes do they appear? Name the position in minutes and frequency in Hz of two distinct energy consumption patterns you find? (report the lowest prominent frequency).

_answer here_

#### 2.3 Step three

Use the function [`firls`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firls.html) from `scipy.filter` to design a FIR low-pass filter that reduces the high-frequency content in the energy data. Use the `plot_filter` function you created earlier to visualise its frequency and phase response. Play around with the parameters of `firls` and see what effect they have on the resulting filter. Then apply your filter to the signal with [`lfilter`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html). Use the function `plot_before_after` given below to visualise the two signals and reuse your `plot_spectrogram` function to display the spectrogram of the signal after filtering.

In [ ]:
def plot_before_after(before_signal, after_signal, samplerate):
    fig = plt.figure()
    timepoints = np.arange(len(before_signal)) / samplerate
    plt.plot(timepoints, before_signal, 'b', timepoints, after_signal, 'g')
    plt.xlabel('time [s]')
    plt.ylabel('amplitude')
    plt.legend(['before', 'after'])
    plt.show()

To get a filter response that's closer to the specification for the same number of taps, we can use an IIR filter. We design one with the [`iirdesign`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.iirdesign.html) function of `scipy.signal`. Many different types of filter can be chosen, but we'll use a Butterworth type (so set `ftype='butter'` as input argument) here.

We'll repeat the same procedure we just did with the FIR filter, but with one important difference. Representing a filter as its transfer function coefficients (numerator `B` and denominator `A`) might be intuitive, but it is not numerically stable. A better way to store a filter is as so-called second-order sections (`SOS`). We need this representation for our IIR filter, otherwise the `lfilter` function will return `NaN`s (try it first if you like). To obtains `SOS`es instead of transfer function coefficients, you need to pass `output=sos` to `iirdesign`. You can use [`sos2ft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.tf2sos.html) to convert the result into `B` and `A` so you can keep using your `plot_filter` function for visualisation (numerical issues won't be a problem for that). Finally, instead of applying `lfilter` to filter the signal, you need to use [`sosfilt`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sosfilt.html).

Can you now isolate some specific frequencies using an IIR band-pass filter?

### 3. Filtering audio signals

Use the same `firls` and `iirdesign` functions as in the previous questions to create high pass and low pass filters and apply them to an audio signal, (e.g. the `audio/carrier.wav` file used in lab 4). Verify your filter effects by listening and by plotting spectrograms.